In [1]:
import numpy as np
import pandas as pd
import scipy
import os
import tensorflow as tf
from Metric_Computation import *

c:\users\iroooon\anaconda3\envs\sci_computing\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\iroooon\anaconda3\envs\sci_computing\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Load the data and Split

In [2]:
# specify the path to read
data_load_path = '../data/intermediate_stock_split/'
# read the data
# train and validation
data_train_X = np.load(data_load_path+'data_X_train.npy')
data_train_X_prev = np.load(data_load_path+'data_X_prev_train.npy')
SP500_train_X = np.load(data_load_path+'SP500_train.npy')
SP500_train_X_prev = np.load(data_load_path+'SP500_prev_train.npy')
value_target_train = np.load(data_load_path+'target_value_train.npy')
value_target_train_prev = np.load(data_load_path+'target_value_prev_train.npy')
gradient_target_train = np.load(data_load_path+'target_gradient_train.npy')
gradient_target_train_prev = np.load(data_load_path+'target_gradient_prev_train.npy')
trend_target_train = np.load(data_load_path+'price_trend_flag_train.npy')
# test
data_test_X = np.load(data_load_path+'data_X_test.npy')
data_test_X_prev = np.load(data_load_path+'data_X_prev_test.npy')
SP500_test_X = np.load(data_load_path+'SP500_test.npy')
SP500_test_X_prev = np.load(data_load_path+'SP500_prev_test.npy')
value_target_test = np.load(data_load_path+'target_value_test.npy')
value_target_test_prev = np.load(data_load_path+'target_value_prev_test.npy')
gradient_target_test = np.load(data_load_path+'target_gradient_test.npy')
gradient_target_test_prev = np.load(data_load_path+'target_gradient_prev_test.npy')
trend_target_test = np.load(data_load_path+'price_trend_flag_test.npy')

In [3]:
# training set: drop instance that contains inf values
valid_series_index_train = np.where((np.sum(np.sum(data_train_X,axis=-1),axis=-1)!=np.inf)&((np.sum(np.sum(data_train_X_prev,axis=-1),axis=-1)!=np.inf)))[0]
data_input_X = data_train_X[valid_series_index_train]
data_input_X_prev = data_train_X_prev[valid_series_index_train]
SP500_input_X = np.reshape(SP500_train_X[valid_series_index_train],[-1,1])
SP500_input_X_prev = np.reshape(SP500_train_X_prev[valid_series_index_train],[-1,1])
value_target = np.reshape(value_target_train[valid_series_index_train],[-1,1])
value_target_prev = np.reshape(value_target_train_prev[valid_series_index_train],[-1,1])
gradient_target = np.reshape(gradient_target_train[valid_series_index_train],[-1,1])
gradient_target_prev = np.reshape(gradient_target_train_prev[valid_series_index_train],[-1,1])
trend_target = np.reshape(trend_target_train[valid_series_index_train],[-1,1])

In [4]:
# testing set: drop instance that contains inf values
valid_series_index_test = np.where((np.sum(np.sum(data_test_X,axis=-1),axis=-1)!=np.inf)&((np.sum(np.sum(data_test_X_prev,axis=-1),axis=-1)!=np.inf)))[0]
data_test_X = data_test_X[valid_series_index_test]
data_test_X_prev = data_test_X_prev[valid_series_index_test]
SP500_test_X = np.reshape(SP500_test_X[valid_series_index_test],[-1,1])
SP500_test_X_prev = np.reshape(SP500_test_X_prev[valid_series_index_test],[-1,1])
value_target_test = np.reshape(value_target_test[valid_series_index_test],[-1,1])
value_target_test_prev = np.reshape(value_target_test_prev[valid_series_index_test],[-1,1])
gradient_target_test = np.reshape(gradient_target_test[valid_series_index_test],[-1,1])
gradient_target_test_prev = np.reshape(gradient_target_test_prev[valid_series_index_test],[-1,1])
trend_target_test = np.reshape(trend_target_test[valid_series_index_test],[-1,1])

In [5]:
# split the training data into 
ind_permutation = np.random.permutation(data_input_X.shape[0])
# training and testing index
training_ind = ind_permutation[:ind_permutation.shape[0]*2//3]
validation_ind = ind_permutation[ind_permutation.shape[0]*2//3:]
# indexing the data
# time series data
# training
data_train_X = data_input_X[training_ind,:,:]
data_train_X_prev = data_input_X_prev[training_ind,:,:]
# validation
data_valid_X = data_input_X[validation_ind,:,:]
data_valid_X_prev = data_input_X[validation_ind,:,:]
# SP500 info
# train
SP500_train_X = np.reshape(SP500_input_X[training_ind],[-1,1])
SP500_train_X_prev = np.reshape(SP500_input_X_prev[training_ind],[-1,1])
# validation
SP500_valid_X = np.reshape(SP500_input_X[validation_ind],[-1,1])
SP500_valid_X_prev = np.reshape(SP500_input_X_prev[validation_ind],[-1,1])
# value prediction (for reference usage only)
# train
value_target_train = np.reshape(value_target[training_ind],[-1,1])
value_target_train_prev = np.reshape(value_target_prev[training_ind],[-1,1])
# validation
value_target_valid = np.reshape(value_target[validation_ind],[-1,1])
value_target_valid_prev = np.reshape(value_target_prev[validation_ind],[-1,1])
# gradient prediction (the real 'labels' for training)
# train
gradient_target_train = np.reshape(gradient_target[training_ind],[-1,1])
gradient_target_train_prev = np.reshape(gradient_target_prev[training_ind],[-1,1])
# validation
gradient_target_valid = np.reshape(gradient_target[validation_ind],[-1,1])
gradient_target_valid_prev = np.reshape(gradient_target_prev[validation_ind],[-1,1])
# trend info
# train
trend_target_train = np.reshape(trend_target[training_ind],[-1,1])
# validation
trend_target_valid = np.reshape(trend_target[validation_ind],[-1,1])

In [6]:
current_and_future_price_train = np.concatenate([value_target_train_prev, value_target_train],axis=1)
current_and_future_price_valid = np.concatenate([value_target_valid_prev, value_target_valid],axis=1)
current_and_future_price_test = np.concatenate([value_target_test_prev, value_target_test],axis=1)

### preceed the trend(class) target to one-hot

In [7]:
def one_hot_encoding_label(label_input, num_class=None):
    '''
    :param label_input: The sparse form of input label (2,0,1,3,0,1,2etc.)
    :param num_class: The number of classes, if keep None, then automatically infer from the given label input
    '''
    # retrieve the number of input data
    nData = label_input.shape[0]
    # reshape the data
    label_input_flat = np.reshape(label_input, [-1])
    if (label_input_flat.shape[0]!=nData):            # which means the input label is not 'mathematically 1-d'
        raise ValueError('The input label must be 1-d mathematically')
    # infer the number of class if input is None
    if num_class is None:
        num_class = (int)(np.amax(label_input)+1)
    # create the return encoded matrx
    one_hot_label_mat = np.zeros([nData, num_class])
    # get a row index to assist the batch-assigning
    row_ind_batch = np.arange(nData)
    # assign '1's to the corresponding positions
    one_hot_label_mat[row_ind_batch, label_input_flat.astype('int')] = 1
    
    return one_hot_label_mat

In [8]:
trend_target_train_input = one_hot_encoding_label(trend_target_train)
trend_target_valid_input = one_hot_encoding_label(trend_target_valid)
trend_target_test_input = one_hot_encoding_label(trend_target_test)

## Evaluation Functions

In [9]:
# confusion metrics
def classification_info_computation(pred_label, true_label, num_class):
    '''
    :param pred_label: the sparse (not one-hot) prediction of labels
    :param true_label: the sparse (not one-hot) ground-truth of labels
    :param num_class: number of classes
    '''
    # flatten the two label arrays if they are not already so
    pred_label = np.reshape(pred_label,[-1])
    true_label = np.reshape(true_label,[-1])
    # initialize the confusion maxtrix array
    class_matrix = np.zeros([num_class, num_class])    # each row is the true labels
    # initialize the precision and recall arrays
    precision_array = np.zeros([num_class])
    recall_array = np.zeros([num_class])
    # fill the confusion-prediction matrix
    for cClass_True in range(num_class):
        # retrieve the current 
        current_cClass_ind = np.where(true_label==cClass_True)[0]
        # retrueve the corresponding predictions
        current_cClass_pred = pred_label[current_cClass_ind]
        # fill the evaluation matrx
        for cClass_Pred in range(num_class):
            cClass_pred_num = np.where(current_cClass_pred==cClass_Pred)[0].shape[0]
            class_matrix[cClass_True, cClass_Pred] = cClass_pred_num
    # fill the precision and recall arrays
    for cClass_True in range(num_class):
        precision_array[cClass_True] = class_matrix[cClass_True,cClass_True]/np.sum(class_matrix[:,cClass_True])
        recall_array[cClass_True] = class_matrix[cClass_True,cClass_True]/np.sum(class_matrix[cClass_True,:])
        
    return class_matrix, precision_array, recall_array

In [10]:
# 'regret' function
def invest_regret_comput(pred_label, true_label):
    '''
    The function to return the 'regret' defined by the real investment scenarios
    :param pred_label: the sparse (not one-hot) prediction of labels
    :param true_label: the sparse (not one-hot) ground-truth of labels
    With the meaning 2=uptrend 1=downtrend 0=non-trend
    Strategy: 
        predict 0: don't buy or sell
        predict 1: sell
        predict 2: buy
    ******************** Truth Table *********************
    | True Label | Predicted Label | Regret |
    |      0     |        0        |   0    |
    |      0     |        1        |   1    |
    |      0     |        2        |   1    |
    |      1     |        0        |   1    |
    |      1     |        1        |   0    |
    |      1     |        2        |   2    |
    |      2     |        0        |   1    |
    |      2     |        1        |   2    |
    |      2     |        2        |   0    |
    '''
    # flatten the two label arrays if they are not already so
    pred_label = np.reshape(pred_label,[-1])
    true_label = np.reshape(true_label,[-1])
    # check if the two arrays are of the same legth
    if pred_label.shape[0]!=true_label.shape[0]:
        raise ValueError('The predicted and the true labels must be in the same length!')
    # placeholder of regret array
    regret_array = np.zeros([pred_label.shape[0]])
    # check the conditions for regret '1'
    one_regret_ind = np.where(((true_label==0)&(pred_label==2))|((true_label==0)&(pred_label==1))|((true_label==2)&(pred_label==0))|((true_label==1)&(pred_label==0)))[0]
    # check the conditions for regret '2'
    two_regret_ind = np.where(((true_label==1)&(pred_label==2))|((true_label==2)&(pred_label==1)))[0]
    # assign regret values to the entries
    regret_array[one_regret_ind] = 1.0
    regret_array[two_regret_ind] = 2.0
    # compute the overall regret
    overall_regret = np.mean(regret_array)
    
    return overall_regret

## Hyper-parameters of the RNN

In [11]:
# rnn cell sequence length and dimensions
nChunck = data_input_X.shape[1]
nDim = data_input_X.shape[2]
# output dimension
nClass = 3
# number of nested RNN stack
nStack_rnn = 5
# rnn dimension
rnn_size = 64
# training hyper-parameter
nEpochs = 100
batch_size = 256
pred_loss_coeff = 1e-3
l2_FC_pen_coeff = 1e-3
fluc_pen_coeff = 1e-4
learning_rate = 1e-4
tol = 1e-3
focal_gamma = 3.0     # parameter for the focal loss
trend_weight_alpha = 2.0  # parameter to encourage output 1/2
# data info for feeding them
nData_train = data_train_X.shape[0]
nData_test = data_test_X.shape[0]

## A 'Erease Graph' here to enable us to re-run things

In [12]:
tf.reset_default_graph()

## Code to save the model

In [13]:
Model_path = '../model/trend_bidirec_RNN_stock_split/'
#Model saving function
def tf_save_model(session):
    if not os.path.exists(Model_path):
        os.makedirs(Model_path)
    saver = tf.train.Saver()
    saver.save(session, Model_path+'Bidirec_RNN.checkpoint')

## define the RNN graph

In [14]:
#Get the x input as float and reshape input
x_time_series = tf.placeholder("float",[None,nChunck,nDim])       # Batch_size * n_chunk * chunk_size
x_time_series_prev = tf.placeholder("float",[None,nChunck,nDim])  # Batch_size * n_chunk * chunk_size
x_SP_data = tf.placeholder("float",[None,1])                      # Batch_size * 1
x_SP_data_prev = tf.placeholder("float",[None,1])                 # Batch_size * 1
x_current_price = tf.placeholder("float",[None,2])                # Batch_size * 2
y = tf.placeholder("float",[None,1])                              # Batch_size * 1
y_prev = tf.placeholder("float",[None,1])                         # Batch_size * 1
y_trend = tf.placeholder("float",[None,nClass])                   # Batch_size * n_classes

In [15]:
def swish_activation(data_input):
    '''
    Swish activation, by Ramachandran et al., Google Brain, 2017
    The activation is found by a RNN-based combinational search, and it has been proved consistently outperforming
        RELU on networks like NASNet-A and Inception-ResNet-v2
    '''
    # define the beta variable as trainable
    para_beta = tf.Variable(tf.random_uniform([1]))
    swish_output = data_input*tf.sigmoid(para_beta*data_input)
    
    return swish_output

In [16]:
def Recurrent_neural_network(x_time_series, x_SP_data, x_time_series_prev, x_SP_data_prev, x_current_price):
    # process the data concatenating the current and previous data
    data_processed = tf.concat([x_time_series,x_time_series_prev],axis=0)
    # split the data into (nChunck * [batch_size, nDim])       
    data_processed = tf.split(data_processed, num_or_size_splits=nChunck, axis=1)  # (nChunck * [batch_size, nDim])
    data_processed = [tf.squeeze(this_time_data, axis=1) for this_time_data in data_processed]
    # define individual GRU cells
    # forward
    stack_GRU_cell_forward = []
    for _ in range(nStack_rnn):
        current_rnn_cell = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.GRUCell(rnn_size),
                                                         input_keep_prob=0.7,
                                                         variational_recurrent=False)# ,
#                                                          input_size=nDim,
#                                                          dtype=tf.float32)
        stack_GRU_cell_forward.append(current_rnn_cell)
    heiarchy_RNN_forward = tf.contrib.rnn.MultiRNNCell(stack_GRU_cell_forward)
    # backward
    stack_GRU_cell_backward = []
    for _ in range(nStack_rnn):
        current_rnn_cell = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.GRUCell(rnn_size),
                                                         input_keep_prob=0.7,
                                                         variational_recurrent=False)#,
#                                                          input_size=nDim,
#                                                          dtype=tf.float32)
        stack_GRU_cell_backward.append(current_rnn_cell)
    heiarchy_RNN_backward = tf.contrib.rnn.MultiRNNCell(stack_GRU_cell_backward)
    # output of the forward-backword RNN
    outputs_all, output_for_final, output_back_final = tf.contrib.rnn.static_bidirectional_rnn(cell_fw=heiarchy_RNN_forward,
                                                                                               cell_bw=heiarchy_RNN_backward,
                                                                                               inputs=data_processed,
                                                                                               dtype=tf.float32)
    # Post-RNN process
    # concatenate the current and prev S&P-500
    x_SP_data_processed = tf.concat([x_SP_data,x_SP_data_prev],axis=0)
    # [nBatch x 2*rnn_stack*rnn_size+1]
    # print(outputs_all[-1].get_shape().as_list())
    out_status_processed = tf.concat([outputs_all[-1], x_SP_data_processed],axis=1)   # [batch_size x (2*rnn_size+1)]
    with tf.name_scope("FC_layers_rate"):
        weight_FC_rate = {'weight_hidden': tf.Variable(tf.random_normal([2*rnn_size+1, 64])),
                     'weight_out': tf.Variable(tf.random_normal([64, 1]))}
    with tf.name_scope("FC_layers_scale"):
        weight_FC_scale = {'weight_hidden': tf.Variable(tf.random_normal([2*rnn_size+1, 64])),
                     'weight_out': tf.Variable(tf.random_normal([64, 1]))}
    with tf.name_scope("FC_layers_class"):
        weight_FC_class = {'weight_hidden_1': tf.Variable(tf.random_normal([2*rnn_size+4, 128])),
                           'weight_hidden_2': tf.Variable(tf.random_normal([128, 64])),
                           'weight_out': tf.Variable(tf.random_normal([64, nClass]))}
    # ***************rate prediction**************
    # fully-connected hidden
    process_output_rate = tf.matmul(out_status_processed, weight_FC_rate["weight_hidden"])    # [batch_size x 64]
    # non-linear activation 
    process_output_rate = swish_activation(process_output_rate)
    process_output_rate = tf.nn.dropout(process_output_rate, keep_prob = 0.7)
    #         process_output = tf.nn.tanh(process_output)
    # process to the final output
    final_output_rate = tf.nn.sigmoid(tf.matmul(process_output_rate, weight_FC_rate["weight_out"]))  # [batch_size x 1]
    # ***************scale prediction**************
    process_output_scale = tf.matmul(out_status_processed, weight_FC_scale["weight_hidden"])
    # non-linear activation 
    process_output_scale = swish_activation(process_output_scale)
    process_output_scale = tf.nn.dropout(process_output_scale, keep_prob = 0.7)
    # out
    # pass to RELU to limit the learning inside the regime of scaling
    final_output_scale = tf.nn.relu(tf.matmul(process_output_scale, weight_FC_scale["weight_out"]))
    # combine them to get the final output
    final_output = tf.multiply(final_output_rate, final_output_scale)
    # split the results to get the prediction for current and previous data
    current_prediction, prev_prediction = tf.split(final_output,num_or_size_splits=2,axis=0)
    # ***************class prediction****************
    out_status_processed_class = tf.concat([tf.split(out_status_processed,num_or_size_splits=2,axis=0)[0],  # only pick the 'current'
                                            x_current_price,
                                            current_prediction],axis=1)
    # layer 1
    process_output_class = tf.matmul(out_status_processed_class, weight_FC_class["weight_hidden_1"])
    process_output_class = swish_activation(process_output_class)
    # layer 2
    process_output_class = tf.matmul(process_output_class, weight_FC_class["weight_hidden_2"])
    process_output_class = swish_activation(process_output_class)
    # layer 3 (out-layer)
    final_output_class = tf.matmul(process_output_class, weight_FC_class["weight_out"])            

    return current_prediction, prev_prediction, final_output_class

In [17]:
def focal_classification_loss(logits, labels, trend_weight):
    '''
    will be resemble the cross-entropy calssification loss, but with the focal loss-decrease term for well-classes 
    examples to tackle the problem of imbalance data.
    :param logits: The [nData * nClass] un-normalised prediction
    :param labels: The [nData * nClass] one-hot true labels
    :param trend_weight: To encourage the network to predict up/down trends. Should be specify
    :return: the loss of [nData] shape
    ''' 
    # get the pobabilistic sotmax output
    soft_max_prediction = tf.nn.softmax(logits, axis=1)
    # process to log(soft_max) output, use this integrated function to faciliate numerical stability
    log_soft_max_prediction = tf.nn.log_softmax(logits, axis=1)
    # compute focal loss
    class_weight = tf.constant([1.0, trend_weight, trend_weight])
    # -sum(y*(1-p)^(gamma)*log(p))  
    focal_loss = -tf.reduce_sum(tf.multiply(tf.multiply(labels,tf.multiply(tf.pow(1-soft_max_prediction,focal_gamma),
                                                                           log_soft_max_prediction)),class_weight),
                                axis=1)
    
    return focal_loss

In [18]:
def train_recurrent_neural_network(x_time_series, x_SP_data, x_time_series_prev, x_SP_data_prev, x_current_price):
    # get the prediction
    grad_pred, grad_pred_prev, class_pred = Recurrent_neural_network(x_time_series, x_SP_data, x_time_series_prev, x_SP_data_prev, x_current_price)
    # mean square loss
    loss_prediction = tf.losses.mean_squared_error(predictions=grad_pred, labels=y)
    # compute the loss of fluctuation
    # use this scheme to back-prop gradient to both sides
    loss_fluctuation = (tf.losses.mean_squared_error(predictions=grad_pred, labels=grad_pred_prev) + 
                        tf.losses.mean_squared_error(predictions=grad_pred_prev, labels=grad_pred))/2
    # mis-classification loss
    # loss_classification = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=class_pred, labels=y_trend))
    # focal loss
    loss_classification = tf.reduce_mean(focal_classification_loss(logits=class_pred, labels=y_trend, trend_weight=trend_weight_alpha))
    # l_2 loss for FC layers
    reg_vars = (tf.trainable_variables(scope='FC_layers_rate') 
                + tf.trainable_variables(scope='FC_layers_scale') 
                + tf.trainable_variables(scope='FC_layers_class'))
    loss_l2_FC = tf.add_n([tf.nn.l2_loss(var) for var in reg_vars])
    # overall loss
    loss = loss_classification + pred_loss_coeff*loss_prediction + l2_FC_pen_coeff*loss_l2_FC + fluc_pen_coeff*loss_fluctuation
    # defineing optimiser
    optimiser_org = tf.train.AdamOptimizer(learning_rate=learning_rate)
    # apply gradient clipping to avoid gradient exploding
    gradients_nn = optimiser_org.compute_gradients(loss)
    clipped_grad = [(tf.clip_by_value(grad, -10., 10.), var) for grad, var in gradients_nn]
    optimiser = optimiser_org.apply_gradients(clipped_grad)
    prev_test_loss = 0
    # evaluation tensor
    label_flatten_tensor = tf.argmax(y_trend, 1)
    logit_flatten_tensor = tf.argmax(class_pred,1)
    acc_tensor = tf.reduce_mean(tf.cast(tf.equal(label_flatten_tensor, logit_flatten_tensor),tf.float32))
    # the training dictionary
    train_dict = {x_time_series: data_train_X,
                  x_time_series_prev: data_train_X_prev,
                  x_SP_data: SP500_train_X,
                  x_SP_data_prev: SP500_train_X_prev,
                  y: gradient_target_train,
                  y_prev: gradient_target_train_prev, 
                  y_trend: trend_target_train_input,
                  x_current_price: current_and_future_price_train}
    # the validation dictionary
    valid_dict = {x_time_series: data_valid_X,
                 x_time_series_prev: data_valid_X_prev,
                 x_SP_data: SP500_valid_X,
                 x_SP_data_prev: SP500_valid_X_prev,
                 y: gradient_target_valid,
                 y_prev: gradient_target_valid_prev, 
                 y_trend: trend_target_valid_input, 
                 x_current_price: current_and_future_price_valid}
    # the test dictionary
    test_dict = {x_time_series: data_test_X,
                 x_time_series_prev: data_test_X_prev,
                 x_SP_data: SP500_test_X,
                 x_SP_data_prev: SP500_test_X_prev,
                 y: gradient_target_test,
                 y_prev: gradient_target_test_prev, 
                 y_trend: trend_target_test_input, 
                 x_current_price: current_and_future_price_test}
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # define the lists to collect the infomation while training
        # train
        train_precision_epoch_collection = []
        train_recall_epoch_collection = []
        train_regret_epoch_collection = []
        # valid
        valid_precision_epoch_collection = []
        valid_recall_epoch_collection = []
        valid_regret_epoch_collection = []
        # test
        test_precision_epoch_collection = []
        test_recall_epoch_collection = []
        test_regret_epoch_collection = []
        for cEpoch in range(nEpochs):
            current_Epoch_Loss = 0
            random_index = np.random.choice(nData_train, size=nData_train)    # Use this to avoid overflow for the derivative index
            for i in range(nData_train//batch_size):
                this_index = random_index[i*batch_size:(i+1)*batch_size]
                current_x_time_series = data_train_X[this_index]
                current_x_time_series_prev = data_train_X_prev[this_index]
                current_x_SP_500 = SP500_train_X[this_index]
                current_x_SP_500_prev = SP500_train_X_prev[this_index]
                current_x_price_info = current_and_future_price_train[this_index]
                current_y_gradient = gradient_target_train[this_index]
                current_y_gradient_prev = gradient_target_train_prev[this_index]
                current_y_trend = trend_target_train_input[this_index]
                #Get how many batches we need for each epoch
                this_feed_dict = {x_time_series: current_x_time_series,
                                  x_time_series_prev: current_x_time_series_prev,
                                  x_SP_data: current_x_SP_500, 
                                  x_SP_data_prev: current_x_SP_500_prev,
                                  y: current_y_gradient, 
                                  y_prev: current_y_gradient_prev,
                                  y_trend: current_y_trend, 
                                  x_current_price: current_x_price_info}
                _, currentloss = sess.run([optimiser,loss], feed_dict = this_feed_dict)
#                 print(currentloss)
#                 print(np.amax(current_x_time_series))
#                 print(np.amin(current_x_time_series))
                current_Epoch_Loss += currentloss
                # break
            print('The',cEpoch+1,'th out of ',nEpochs,'Epochs in total has finished and loss in this epoch is',current_Epoch_Loss)
            # Check the classification loss for training and testing data
            train_data_loss = loss_classification.eval(feed_dict=train_dict)
            valid_data_loss = loss_classification.eval(feed_dict=valid_dict)
            test_data_loss = loss_classification.eval(feed_dict=test_dict)
            print('The classification loss: training: ',train_data_loss, ' validation: ', valid_data_loss, ' test: ', test_data_loss)
            # Check the regression loss for training and testing data
            train_data_loss_reg = loss_prediction.eval(feed_dict=train_dict)
            valid_data_loss_reg = loss_prediction.eval(feed_dict=valid_dict)
            test_data_loss_reg = loss_prediction.eval(feed_dict=test_dict)
            print('The regression loss: training: ',train_data_loss_reg, ' validation: ', valid_data_loss_reg, ' test: ', test_data_loss_reg)
            # Check the fluctuation loss for training and testing data
            train_data_loss_fluc = loss_fluctuation.eval(feed_dict=train_dict)
            valid_data_loss_fluc = loss_fluctuation.eval(feed_dict=valid_dict)
            test_data_loss_fluc = loss_fluctuation.eval(feed_dict=test_dict)
            print('The fluctuation loss: training: ',train_data_loss_fluc, ' validation: ', valid_data_loss_fluc, ' test: ', test_data_loss_fluc)
            # Also check the current classification accuracy
            train_data_accuracy = acc_tensor.eval(feed_dict=train_dict)
            valid_data_accuracy = acc_tensor.eval(feed_dict=valid_dict)
            test_data_accuracy = acc_tensor.eval(feed_dict=test_dict)
            # evluation and accuracy
            # flattened predictions
            # train
            train_logit_flatten_numerical = logit_flatten_tensor.eval(train_dict)
            train_label_flatten_numerical = label_flatten_tensor.eval(train_dict)
            # validation
            valid_logit_flatten_numerical = logit_flatten_tensor.eval(valid_dict)
            valid_label_flatten_numerical = label_flatten_tensor.eval(valid_dict)
            # test
            test_logit_flatten_numerical = logit_flatten_tensor.eval(test_dict)
            test_label_flatten_numerical = label_flatten_tensor.eval(test_dict)
            # train
            class_matrix_train, precision_array_train, recall_array_train = classification_info_computation(pred_label = train_logit_flatten_numerical, 
                                                                                                            true_label = train_label_flatten_numerical, 
                                                                                                            num_class=3)
            down_mis_up_rate_train = class_matrix_train[2,1]/np.sum(class_matrix_train[:,1])
            up_mis_down_rate_train = class_matrix_train[1,2]/np.sum(class_matrix_train[:,2])
            precision_downtrend_adjusted_train = precision_array_train[1] - down_mis_up_rate_train
            precision_uptrend_adjuest_train = precision_array_train[2] - up_mis_down_rate_train
            # validation
            class_matrix_valid, precision_array_valid, recall_array_valid = classification_info_computation(pred_label = valid_logit_flatten_numerical, 
                                                                                                            true_label = valid_label_flatten_numerical, 
                                                                                                            num_class=3)
            down_mis_up_rate_valid = class_matrix_valid[2,1]/np.sum(class_matrix_valid[:,1])
            up_mis_down_rate_valid = class_matrix_valid[1,2]/np.sum(class_matrix_valid[:,2])
            precision_downtrend_adjusted_valid = precision_array_valid[1] - down_mis_up_rate_valid
            precision_uptrend_adjuest_valid = precision_array_valid[2] - up_mis_down_rate_valid
            # test
            class_matrix_test, precision_array_test, recall_array_test = classification_info_computation(pred_label = test_logit_flatten_numerical, 
                                                                                                         true_label = test_label_flatten_numerical, 
                                                                                                         num_class=3)
            down_mis_up_rate_test = class_matrix_test[2,1]/np.sum(class_matrix_test[:,1])
            up_mis_down_rate_test = class_matrix_test[1,2]/np.sum(class_matrix_test[:,2])
            precision_downtrend_adjusted_test = precision_array_test[1] - down_mis_up_rate_test
            precision_uptrend_adjuest_test = precision_array_test[2] - up_mis_down_rate_test
            # training and testing regret
            regret_epoch_train = invest_regret_comput(pred_label = train_logit_flatten_numerical,
                                                      true_label = train_label_flatten_numerical)
            regret_epoch_valid = invest_regret_comput(pred_label = valid_logit_flatten_numerical,
                                                      true_label = valid_label_flatten_numerical)
            regret_epoch_test = invest_regret_comput(pred_label = test_logit_flatten_numerical,
                                                     true_label = test_label_flatten_numerical)
            # collect the information
            # train
            train_precision_epoch_collection.append(precision_array_train) 
            train_recall_epoch_collection.append(recall_array_train)
            train_regret_epoch_collection.append(regret_epoch_train)
            # validation
            valid_precision_epoch_collection.append(precision_array_valid)
            valid_recall_epoch_collection.append(recall_array_valid)
            valid_regret_epoch_collection.append(regret_epoch_valid)
            # test
            test_precision_epoch_collection.append(precision_array_test)
            test_recall_epoch_collection.append(recall_array_test)
            test_regret_epoch_collection.append(regret_epoch_test)
            # print out the information
            print('The classification accuracy, train: ', train_data_accuracy, ' validation: ', valid_data_accuracy,' test: ', test_data_accuracy)
            print('The train regret of the current epoch is: ', regret_epoch_train)
            print('The test regret of the current epoch is: ', regret_epoch_test)
            print('Recall Info:')
            print('On the training data, the recall of Non-trend:', recall_array_train[0], ' Downtrend:', recall_array_train[1], ' Uptrend:', recall_array_train[2])
            print('On the validation data, the recall of Non-trend:', recall_array_valid[0], ' Downtrend:', recall_array_valid[1], ' Uptrend:', recall_array_valid[2])
            print('On the testing data, the recall of Non-trend:', recall_array_test[0], ' Downtrend:', recall_array_test[1], ' Uptrend:', recall_array_test[2])
            print('Precision Info:')
            print('On the training data, the precision of Non-trend:', precision_array_train[0], ' Downtrend:', precision_array_train[1], ' Uptrend:', precision_array_train[2])
            print('On the validation data, the precision of Non-trend:', precision_array_valid[0], ' Downtrend:', precision_array_valid[1], ' Uptrend:', precision_array_valid[2])
            print('On the testing data, the precision of Non-trend:', precision_array_test[0], ' Downtrend:', precision_array_test[1], ' Uptrend:', precision_array_test[2])
            print('Adjusted Precision:')
            print('On the training data, Adjusted Downtrend precision: ', precision_downtrend_adjusted_train, 'Adjusted Uptrend precision: ', precision_uptrend_adjuest_train)
            print('On the validation data, Adjusted Downtrend precision: ', precision_downtrend_adjusted_valid, 'Adjusted Uptrend precision: ', precision_uptrend_adjuest_valid)
            print('On the testing data, Adjusted Downtrend precision: ', precision_downtrend_adjusted_test, 'Adjusted Uptrend precision: ', precision_uptrend_adjuest_test)
            print('**************************I\'m the Divider*****************************')
            if(abs((prev_test_loss-currentloss)/currentloss)<=tol):
                print('Network stucked to local minimum!')
                tf_save_model(sess)
                break
            # recall of three trends to determine convergence
            if (precision_array_valid[0]>=0.50)and(recall_array_valid[1]>=0.15)and(recall_array_valid[2]>=0.15):
                save_class_matrix_train = class_matrix_train[:]
                save_class_matrix_valid = class_matrix_valid[:]
                save_class_matrix_test = class_matrix_test[:]
                tf_save_model(sess)
                if regret_epoch_train<=0.60:
                    print('Network converged!')
                    break
            # precision of the non-trend as the convergence critaria
            downtrend_satis_condition = abs(precision_array_valid[1]-precision_downtrend_adjusted_valid)/abs(precision_array_valid[1])<1
            uptrend_satis_condition = abs(precision_array_valid[2]-precision_uptrend_adjuest_valid)/abs(precision_array_valid[2])<1
            if (recall_array_valid[0]>=0.50)and((downtrend_satis_condition)or(uptrend_satis_condition)):
                save_class_matrix_train = class_matrix_train[:]
                save_class_matrix_valid = class_matrix_valid[:]
                save_class_matrix_test = class_matrix_test[:]
                tf_save_model(sess)
                if regret_epoch_train<=0.60:
                    print('Network converged!')
                    break
            if cEpoch == (nEpochs-1):
                print('Network failed to converge!')
            prev_test_loss = currentloss
    # make the lists as arrays
    # train
    train_precision_epoch_array = np.array(train_precision_epoch_collection)
    train_recall_epoch_array = np.array(train_recall_epoch_collection)
    train_regret_epoch_array = np.array(train_regret_epoch_collection)
    # put them into one list to save 
    train_epoch_wise_info = [train_precision_epoch_array, train_recall_epoch_array, train_regret_epoch_array, save_class_matrix_train]
    # validation
    valid_precision_epoch_array = np.array(valid_precision_epoch_collection)
    valid_recall_epoch_array = np.array(valid_recall_epoch_collection)
    valid_regret_epoch_array = np.array(valid_regret_epoch_collection)
    # put them into one list to save 
    valid_epoch_wise_info = [valid_precision_epoch_array, valid_recall_epoch_array, valid_regret_epoch_array, save_class_matrix_valid]
    # test
    test_precision_epoch_array = np.array(test_precision_epoch_collection)
    test_recall_epoch_array = np.array(test_recall_epoch_collection)
    test_regret_epoch_array = np.array(test_regret_epoch_collection)
    # put them into one list to save
    test_epoch_wise_info = [test_precision_epoch_array, test_recall_epoch_array, test_regret_epoch_array, save_class_matrix_test]
    
    return train_epoch_wise_info, valid_epoch_wise_info, test_epoch_wise_info

In [19]:
train_epoch_wise_info, valid_epoch_wise_info, test_epoch_wise_info = train_recurrent_neural_network(x_time_series, x_SP_data, x_time_series_prev, x_SP_data_prev, x_current_price)

The 1 th out of  100 Epochs in total has finished and loss in this epoch is 65572170.26953125
The classification loss: training:  7824.9663  validation:  7623.508  test:  6333.705
The regression loss: training:  53565464.0  validation:  53431870.0  test:  54099544.0
The fluctuation loss: training:  61058990.0  validation:  60667680.0  test:  61588476.0
The classification accuracy, train:  0.38951525  validation:  0.38961986  test:  0.38212466
The train regret of the current epoch is:  0.708711847814364
The test regret of the current epoch is:  0.7171826754568973
Recall Info:
On the training data, the recall of Non-trend: 0.404248973969354  Downtrend: 0.11492482129652452  Uptrend: 0.48651599197450723
On the validation data, the recall of Non-trend: 0.40416441165579514  Downtrend: 0.11772732911549436  Uptrend: 0.4857025904917841
On the testing data, the recall of Non-trend: 0.3631721670897761  Downtrend: 0.12059477812902471  Uptrend: 0.5598444045481747
Precision Info:
On the training dat

The 6 th out of  100 Epochs in total has finished and loss in this epoch is 119775.55953216553
The classification loss: training:  148.59787  validation:  149.88722  test:  109.70625
The regression loss: training:  26542.445  validation:  27056.064  test:  18540.139
The fluctuation loss: training:  45397.266  validation:  39238.973  test:  35953.27
The classification accuracy, train:  0.45794237  validation:  0.4553087  test:  0.4546107
The train regret of the current epoch is:  0.6141350788554552
The test regret of the current epoch is:  0.6128515396812151
Recall Info:
On the training data, the recall of Non-trend: 0.675277276793956  Downtrend: 0.19965491742666996  Uptrend: 0.12858491679452377
On the validation data, the recall of Non-trend: 0.6749081379781809  Downtrend: 0.19488897159161395  Uptrend: 0.12274136015437694
On the testing data, the recall of Non-trend: 0.6769559196861297  Downtrend: 0.17726261561878  Uptrend: 0.136983842010772
Precision Info:
On the training data, the pr

The 11 th out of  100 Epochs in total has finished and loss in this epoch is 72015.8387298584
The classification loss: training:  105.5665  validation:  104.66159  test:  80.88751
The regression loss: training:  2420.5068  validation:  1499.9949  test:  1070.9803
The fluctuation loss: training:  3749.9524  validation:  2078.1064  test:  1812.0166
The classification accuracy, train:  0.4561172  validation:  0.4567854  test:  0.45022115
The train regret of the current epoch is:  0.6095057949011506
The test regret of the current epoch is:  0.6187766001835934
Recall Info:
On the training data, the recall of Non-trend: 0.5828564125140236  Downtrend: 0.054288883411387726  Uptrend: 0.38625634367992445
On the validation data, the recall of Non-trend: 0.5823909761586008  Downtrend: 0.055576231236819255  Uptrend: 0.38582539032805097
On the testing data, the recall of Non-trend: 0.5737075928917609  Downtrend: 0.048354993560473014  Uptrend: 0.3964093357271095
Precision Info:
On the training data, 

In [20]:
save_path = '../results_stock_split/'
# make the path if it does not exist yet
if not os.path.exists(save_path):
    os.makedirs(save_path)
# save training
np.save(save_path+'train_precision_epoch_array.npy',train_epoch_wise_info[0])
np.save(save_path+'train_recall_epoch_array.npy',train_epoch_wise_info[1]) 
np.save(save_path+'train_regret_epoch_array.npy',train_epoch_wise_info[2]) 
np.save(save_path+'save_class_matrix_train.npy',train_epoch_wise_info[3]) 
# save validation
np.save(save_path+'valid_precision_epoch_array.npy',valid_epoch_wise_info[0])
np.save(save_path+'valid_recall_epoch_array.npy',valid_epoch_wise_info[1]) 
np.save(save_path+'valid_regret_epoch_array.npy',valid_epoch_wise_info[2]) 
np.save(save_path+'save_class_matrix_valid.npy',valid_epoch_wise_info[3]) 
# save the testing
np.save(save_path+'test_precision_epoch_array.npy',test_epoch_wise_info[0])
np.save(save_path+'test_recall_epoch_array.npy',test_epoch_wise_info[1]) 
np.save(save_path+'test_regret_epoch_array.npy',test_epoch_wise_info[2]) 
np.save(save_path+'save_class_matrix_test.npy',test_epoch_wise_info[3])